________
A notebook for visualizing missing data patterns in hourly solar PV data. 

Gaps at night (between sunset and sunrise), first, are imputed with 0. So, they are not presented in the final plot.
________


In [ ]:
import pandas as pd
import numpy as np

In [21]:
# Data from sites
house_M = pd.read_excel("house M 5 min production.xlsx")
melania = pd.read_excel("melania 5 min production.xlsx")

In [22]:
# Apply rolling average and pick data from the end of every hour

def rolling_average_per_hour_interval(df):
    
    df['UTC'] = pd.to_datetime(df['UTC'])

    df_rolling_mean = df.loc[:, ['UTC', 'power [W]']].set_index('UTC').rolling('1h', min_periods = 1).mean()
    df_rolling_mean['UTC'] = df_rolling_mean.index
    df_rolling_mean = df_rolling_mean.reset_index(drop=True)
               
    for i in range(df_rolling_mean.shape[0]):
        #print(df_rolling_mean.UTC[i].hour, df_rolling_mean.UTC[i].minute, df_rolling_mean[i].UTC.second)
        if df_rolling_mean.UTC[i].hour == 1 and df_rolling_mean.UTC[i].minute == df_rolling_mean.UTC[i].second == 0:
            index_start = i
            break
            
                
    for i in range(df_rolling_mean.shape[0]-1, -1, -1):
        if df_rolling_mean.UTC[i].hour == df_rolling_mean.UTC[i].minute == df_rolling_mean.UTC[i].second == 0:
            index_stop = i
            break
            
    return df_rolling_mean.iloc[index_start:(index_stop+1)][::12].reset_index(drop = True)

In [23]:
house_M_hourly = rolling_average_per_hour_interval(house_M)
melania_hourly = rolling_average_per_hour_interval(melania)

In [24]:
import datetime
from suntime import Sun

def get_sunrise_sunset(df, latitude, longitude):
    
    period_start = df.loc[:, 'UTC'].iloc[0]
    year_start, month_start, day_start = period_start.year, period_start.month, period_start.day

    period_end = df.loc[:, 'UTC'].iloc[df.shape[0]-1]
    year_end, month_end, day_end = period_end.year, period_end.month, period_end.day
    
    all_timestamps_utc = pd.date_range(start = datetime.date(year_start, month_start, day_start), 
                                   end = datetime.date(year_end, month_end, day_end), freq="1D")
    
    sun = Sun(latitude, longitude)
    sunrises_sunsets = dict()
    
    for day in all_timestamps_utc:
        sunrises_sunsets[day.strftime('%Y-%m-%d %H:%M:%S').split()[0]] = {
                                                                            'sunrise': sun.get_sunrise_time(day).strftime('%Y-%m-%d %H:%M:%S'),
                                                                            'sunset': sun.get_sunset_time(day).strftime('%Y-%m-%d %H:%M:%S')
                                                                        }

    return sunrises_sunsets

In [25]:
latitude_Vuorela, longitude_Vuorela = 62.97999, 27.64920
sunrises_sunsets_Vuorela = get_sunrise_sunset(house_M_hourly, latitude_Vuorela, longitude_Vuorela)

latitude_Savilahti, longitude_Savilahti = 62.89216, 27.63362
sunrises_sunsets_Savilahti = get_sunrise_sunset(melania_hourly, latitude_Savilahti, longitude_Savilahti)

In [26]:
# Filling gaps at night (between sunset and sunrise with 0s)
def check_sun(x, sunrises_sunsets):
    '''
    Check if a timestamp between sunrise and sunset
    '''
    if (x >= pd.to_datetime(sunrises_sunsets[x.strftime('%Y-%m-%d %H:%M:%S').split()[0]]['sunrise']) and 
        x <= pd.to_datetime(sunrises_sunsets[x.strftime('%Y-%m-%d %H:%M:%S').split()[0]]['sunset'])):
        return 1
    else:
        return 0

def add_zeros_night(df, sunrise_sunset):
    '''
    Fill gaps with 0 if 'sun is up' == 0
    '''
    df['sun is up'] = df.UTC.apply(lambda x: check_sun(x, sunrise_sunset))
    df.loc[df['power [W]'].isna() & (df['sun is up'] == 0), 'power [W]'] = 0

In [27]:
add_zeros_night(house_M_hourly, sunrises_sunsets_Vuorela)
add_zeros_night(melania_hourly, sunrises_sunsets_Savilahti)

In [29]:
def gaps_counter(df):
    df['date'] = df.UTC.apply(lambda x: x.strftime('%Y-%m-%d %H:%M:%S').split()[0])
    df['hour'] = df.UTC.apply(lambda x: x.hour)
    df['gap'] = df['power [W]'].isna().astype(int)
    
    return df.groupby(by=['date']).sum()

In [32]:
gaps_counter(house_M_hourly)
gaps_counter(melania_hourly)

,power [W],sun is up,hour,gap
date,,,,
2016-09-23,10046.426515,13,276,1
2016-09-24,13798.070833,13,276,1
2016-09-25,11540.997727,11,276,0
2016-09-26,14169.600000,11,276,0
2016-09-27,30303.826111,11,276,0
...,...,...,...,...
2021-05-26,67452.628333,19,276,0
2021-05-27,46553.466667,19,276,1
2021-05-28,94527.697222,19,276,1


In [35]:
from bokeh.io import output_file, show
from bokeh.models import ColumnDataSource
from bokeh.plotting import figure
from bokeh.transform import jitter
from bokeh.models import BoxAnnotation
from bokeh.models.widgets import Panel, Tabs
from bokeh.layouts import layout
from bokeh.models import HoverTool


output_file("measured_power&missing_values.html", mode='inline')

HOURS = ['01:00', '02:00', '03:00', '04:00', '05:00', '06:00', '07:00', '08:00', '09:00', '10:00', '11:00', '12:00',
       '13:00', '14:00', '15:00', '16:00', '17:00', '18:00', '19:00', '20:00', '21:00', '22:00', '23:00', '24:00']

def hours_transformer(x):
    if x == '0':
        x = '24'
        
    if len(x) == 1:
        x = '0' + x
        
    return x + ':00'

def data_creator_gaps(df):
    data = df.loc[df.gap == 1, ['date', 'hour']].reset_index(drop=True)
    data['date'] = pd.to_datetime(data['date'])
    data['hour'] = data['hour'].astype(str)
    data['hour'] = data['hour'].apply(lambda x: hours_transformer(x))
    
    return data

def data_creator_power(df):
    data = df.loc[:, ['power [W]', 'UTC']]
    return data


def figure_missing_values(df, data, site):
    source = ColumnDataSource(data)
    x_range = (pd.to_datetime(df.date[0], format = '%Y-%m-%d'), 
           pd.to_datetime(df.date[df.shape[0]-1], format = '%Y-%m-%d'))

    p = figure(plot_width=1500, plot_height=275, y_range=HOURS, x_range=x_range, x_axis_type='datetime', #toolbar_location=None,
           title=site)
    p.background_fill_color = "grey"
    p.background_fill_alpha = 0.05
    p.circle(x='date', y='hour', width=0.25,  source=source, alpha=0.2, color = 'brown')
    # Highlight the region of interest
    box = BoxAnnotation(left=datetime.datetime(2020, 5, 5), right = datetime.datetime(2020, 11, 5), fill_alpha=0.1, fill_color='green')
    p.add_layout(box)
    
    return p

def figure_power_measured(df, data, site):
    source = ColumnDataSource(data)
    x_range = (pd.to_datetime(df.date[0], format = '%Y-%m-%d'), 
           pd.to_datetime(df.date[df.shape[0]-1], format = '%Y-%m-%d'))

    p = figure(plot_width=1500, plot_height=275, x_range=x_range, x_axis_type='datetime', #toolbar_location=None,
           title=site)
    p.background_fill_color = "grey"
    p.background_fill_alpha = 0.05
    p.line(x='UTC', y='power [W]', width=0.5,  source=source, alpha=0.5, color = 'brown')
    # Highlight the region of interest
    box = BoxAnnotation(left=datetime.datetime(2020, 5, 5), right = datetime.datetime(2020, 11, 5), fill_alpha=0.1, fill_color='green')
    p.add_layout(box)
    hover_tool = HoverTool(tooltips=[('UTC', '@UTC{%Y-%m-%d %H:%M}'), ('power [W]', '@{power [W]}{0,0.00}')],
                       formatters={'@UTC': 'datetime'},)
    p.add_tools(hover_tool)
    
    return p


data_power_M = data_creator_power(house_M_hourly)
p11 = figure_power_measured(house_M_hourly, data_power_M, "Vuorela")
data_power_melania = data_creator_power(melania_hourly)
p21 = figure_power_measured(melania_hourly, data_power_melania, "Savilahti")

data_miss_val_M = data_creator_gaps(house_M_hourly)
p12 = figure_missing_values(house_M_hourly, data_miss_val_M, "Vuorela")
data_miss_val_melania = data_creator_gaps(melania_hourly)
p22 = figure_missing_values(melania_hourly, data_miss_val_melania, "Savilahti")

tabs_1 = Tabs(tabs=[Panel(child=p11, title="Power [W]"), Panel(child=p12, title="Missing values")])
tabs_2 = Tabs(tabs=[Panel(child=p21, title="Power [W]"), Panel(child=p22, title="Missing values")])


show(layout([tabs_1, tabs_2]))